In [ ]:
import plotly.graph_objects as go
from dataTable import *
from weaponData import *

settings = et.parse("settings.xml").getroot()
generalSettings = settings.find("general")
graphSettings = settings.find("graphSettings")
health = int(generalSettings.get("health"))
interpolation = generalSettings.get("interpolation")
def plotWeapon(weaponToPlot):
    xAxis = graphSettings.get("xAxis")
    yAxis = graphSettings.get("yAxis")
    hoverFields = graphSettings.get("hoverFields").split(',')
    global fig
    id = 0
    try:
        id = int(weaponToPlot)
    except:
        # 把weaponToPlot从武器数据转换到ID
        id = getIDByName(weaponToPlot)
    weapon = WeaponData(id)
    killData = weapon.getKillData(health)
    x=killData[xAxis].tolist()
    y=killData[yAxis].tolist()
    hoverText = []
    # 遍历X轴的所有数据
    for i in x:
        text = ''
        # 找到浮动窗口中对应当前X的数值
        for j in hoverFields:
            text += j + ': ' + str(killData[killData[xAxis] == i][j].iloc[0]) + '<br>'
        hoverText.append(text)
    fig.add_trace(go.Scatter(x=x, y=y, line_shape = interpolation, name = weapon.name + " (ID: {})".format(str(weapon.id)),
                             hovertemplate = 
                                 '<br><b>' + yAxis + ': %{y}</b><br>' + 
                                 '<b>' + xAxis + ': %{x}</b><br>' +
                                 '%{text}', 
                                 text = hoverText,
                             ))
    fig.update_layout(hovermode='x unified', xaxis_title=xAxis, yaxis_title=yAxis, width=1000, height=600,)

# 注意事项：
如果你对任何此文件之外的文件进行了修改（代码、数据、settings.xml设置文件等），记得在运行之前点Restart重启kernel，否则数据不会更新。

In [2]:
fig = go.FigureWidget()
# 在此声明要绘制哪些武器
plotWeapon('M4')
plotWeapon('AK47')
plotWeapon('S1897')
plotWeapon('MP5')
plotWeapon('Baret')
fig

FigureWidget({
    'data': [{'hovertemplate': '<br><b>TTK: %{y}</b><br><b>distance: %{x}</b><br>%{text}',
              'line': {'shape': 'vh'},
              'name': 'M4: 0',
              'text': [STK: 4.0<br>damage: 30.0 (30.0 * 1)<br>fireInterval:
                       80<br>fireRate: 1.0<br>DPS: 0.38<br>, STK: 4.0<br>damage:
                       30.0 (30.0 * 1)<br>fireInterval: 80<br>fireRate: 1.0<br>DPS:
                       0.38<br>, STK: 5.0<br>damage: 24.0 (24.0 *
                       1)<br>fireInterval: 80<br>fireRate: 1.0<br>DPS: 0.3<br>,
                       STK: 5.0<br>damage: 21.0 (21.0 * 1)<br>fireInterval:
                       80<br>fireRate: 1.0<br>DPS: 0.26<br>],
              'type': 'scatter',
              'uid': '5160fd58-107b-404d-9f77-e2ae5026ba17',
              'x': [0.0, 10.0, 25.0, 60.0],
              'y': [240.0, 240.0, 320.0, 320.0]},
             {'hovertemplate': '<br><b>TTK: %{y}</b><br><b>distance: %{x}</b><br>%{text}',
              'line'